In [1]:
import nflreadpy as nfl
from datetime import datetime
import pandas as pd

import numpy as np

In [2]:
seasons_to_load = list(range(2024, 2014, -1))

schedules_polars = nfl.load_schedules(seasons=seasons_to_load)

In [3]:
schedules_df =  schedules_polars.to_pandas()
del schedules_polars

schedules_df.columns

Index(['game_id', 'season', 'game_type', 'week', 'gameday', 'weekday',
       'gametime', 'away_team', 'away_score', 'home_team', 'home_score',
       'location', 'result', 'total', 'overtime', 'old_game_id', 'gsis',
       'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'away_rest', 'home_rest',
       'away_moneyline', 'home_moneyline', 'spread_line', 'away_spread_odds',
       'home_spread_odds', 'total_line', 'under_odds', 'over_odds', 'div_game',
       'roof', 'surface', 'temp', 'wind', 'away_qb_id', 'home_qb_id',
       'away_qb_name', 'home_qb_name', 'away_coach', 'home_coach', 'referee',
       'stadium_id', 'stadium'],
      dtype='object')

In [4]:
schedules_df = schedules_df[schedules_df['game_type'] == 'REG']

schedule_columns_of_interest = [
    'game_id',
    'season',
    'week',
    'gameday',
    # 'gametime',
    'away_team',
    'away_score',
    'home_team',
    'home_score',
    'location',
    'roof',
    'temp',
    'wind',
    'stadium_id',
    'stadium'
]

schedules_df_filtered = schedules_df[schedule_columns_of_interest]
schedules_df_filtered.head()

,game_id,season,week,gameday,away_team,away_score,home_team,home_score,location,roof,temp,wind,stadium_id,stadium
0,2015_01_PIT_NE,2015,1,2015-09-10,PIT,21,NE,28,Home,outdoors,65.0,7.0,BOS00,Gillette Stadium
1,2015_01_IND_BUF,2015,1,2015-09-13,IND,14,BUF,27,Home,outdoors,56.0,15.0,BUF00,Ralph Wilson Stadium
2,2015_01_GB_CHI,2015,1,2015-09-13,GB,31,CHI,23,Home,outdoors,72.0,11.0,CHI98,Soldier Field
3,2015_01_KC_HOU,2015,1,2015-09-13,KC,27,HOU,20,Home,closed,NaN,NaN,HOU00,NRG Stadium
4,2015_01_CAR_JAX,2015,1,2015-09-13,CAR,20,JAX,9,Home,outdoors,77.0,7.0,JAX00,EverBank Field


In [5]:
# This method for converting a date to a week number is copied from 
#   Adrian's implementation in the weather processing code. This is to
#   ensure consistency between the two datasets.
def convert_to_weeknum(x):
    dt = x.to_pydatetime(x)
    weeknum = ((x - datetime(x.year,1,1)).days // 7) + 1
    return weeknum

In [6]:
schedules_df_filtered['calendar_week'] = schedules_df_filtered['gameday'].apply(datetime.strptime, args=('%Y-%m-%d',)).apply(convert_to_weeknum)
display(schedules_df_filtered[['gameday', 'calendar_week']].sample(5))
schedules_df_filtered.drop(['gameday'], axis=1, inplace=True)

C:\Users\Adrian\AppData\Local\Temp\ipykernel_30612\1228534522.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedules_df_filtered['calendar_week'] = schedules_df_filtered['gameday'].apply(datetime.strptime, args=('%Y-%m-%d',)).apply(convert_to_weeknum)


,gameday,calendar_week
2434,2024-01-07,1
2660,2024-12-08,49
2532,2024-10-06,40
1665,2021-10-03,40
311,2016-09-25,39


C:\Users\Adrian\AppData\Local\Temp\ipykernel_30612\1228534522.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedules_df_filtered.drop(['gameday'], axis=1, inplace=True)


In [7]:
schedules_df_filtered.loc[schedules_df_filtered['temp'].isna() & schedules_df_filtered['roof'].isin(['closed', 'dome']), 'temp'] = 70.0
schedules_df_filtered.loc[schedules_df_filtered['wind'].isna() & schedules_df_filtered['roof'].isin(['closed', 'dome']), 'wind'] = 0.0

schedules_df_filtered.dropna(subset=['temp', 'wind'], inplace=True)

C:\Users\Adrian\AppData\Local\Temp\ipykernel_30612\2830459347.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  schedules_df_filtered.dropna(subset=['temp', 'wind'], inplace=True)


In [8]:
schedules_df_filtered.head()

,game_id,season,week,away_team,away_score,home_team,home_score,location,roof,temp,wind,stadium_id,stadium,calendar_week
0,2015_01_PIT_NE,2015,1,PIT,21,NE,28,Home,outdoors,65.0,7.0,BOS00,Gillette Stadium,37
1,2015_01_IND_BUF,2015,1,IND,14,BUF,27,Home,outdoors,56.0,15.0,BUF00,Ralph Wilson Stadium,37
2,2015_01_GB_CHI,2015,1,GB,31,CHI,23,Home,outdoors,72.0,11.0,CHI98,Soldier Field,37
3,2015_01_KC_HOU,2015,1,KC,27,HOU,20,Home,closed,70.0,0.0,HOU00,NRG Stadium,37
4,2015_01_CAR_JAX,2015,1,CAR,20,JAX,9,Home,outdoors,77.0,7.0,JAX00,EverBank Field,37


In [14]:
split_teams_df = pd.concat([
    schedules_df_filtered.assign(
        focus_team=schedules_df_filtered['home_team'],
        focus_score=schedules_df_filtered['home_score'],
        did_win=schedules_df_filtered['home_score'] > schedules_df_filtered['away_score'],
        focus_temp_delta=np.nan # TODO methodology decision. expanded on below #EA setting to na so temp delta is calc'd minus the avg

    ),
    schedules_df_filtered.assign(
        focus_team=schedules_df_filtered['away_team'],
        focus_score=schedules_df_filtered['away_score'],
        did_win=schedules_df_filtered['away_score'] > schedules_df_filtered['home_score'],
    )
])

split_teams_df[(split_teams_df['week'] == 1) & (split_teams_df['season'] == 2015)].sort_values('stadium').head(8)

,game_id,season,week,away_team,away_score,home_team,home_score,location,roof,temp,wind,stadium_id,stadium,calendar_week,focus_team,focus_score,did_win,focus_temp_delta
13,2015_01_NYG_DAL,2015,1,NYG,26,DAL,27,Home,closed,70.0,0.0,DAL00,AT&T Stadium,37,NYG,26,False,NaN
13,2015_01_NYG_DAL,2015,1,NYG,26,DAL,27,Home,closed,70.0,0.0,DAL00,AT&T Stadium,37,DAL,27,True,NaN
6,2015_01_SEA_STL,2015,1,SEA,31,STL,34,Home,dome,70.0,0.0,STL00,Edward Jones Dome,37,SEA,31,False,NaN
6,2015_01_SEA_STL,2015,1,SEA,31,STL,34,Home,dome,70.0,0.0,STL00,Edward Jones Dome,37,STL,34,True,NaN
4,2015_01_CAR_JAX,2015,1,CAR,20,JAX,9,Home,outdoors,77.0,7.0,JAX00,EverBank Field,37,JAX,9,False,NaN
4,2015_01_CAR_JAX,2015,1,CAR,20,JAX,9,Home,outdoors,77.0,7.0,JAX00,EverBank Field,37,CAR,20,True,NaN
7,2015_01_MIA_WAS,2015,1,MIA,17,WAS,10,Home,outdoors,69.0,10.0,WAS00,FedExField,37,MIA,17,True,NaN
7,2015_01_MIA_WAS,2015,1,MIA,17,WAS,10,Home,outdoors,69.0,10.0,WAS00,FedExField,37,WAS,10,False,NaN


In [15]:
split_teams_df = split_teams_df[[
    'game_id',
    'season',
    'week',
    'temp',
    'wind',
    'calendar_week',
    'focus_team',
    'focus_score',
    'focus_temp_delta',
    'did_win',
]]

split_teams_df

,game_id,season,week,temp,wind,calendar_week,focus_team,focus_score,focus_temp_delta,did_win
0,2015_01_PIT_NE,2015,1,65.0,7.0,37,NE,28,NaN,True
1,2015_01_IND_BUF,2015,1,56.0,15.0,37,BUF,27,NaN,True
2,2015_01_GB_CHI,2015,1,72.0,11.0,37,CHI,23,NaN,False
3,2015_01_KC_HOU,2015,1,70.0,0.0,37,HOU,20,NaN,False
4,2015_01_CAR_JAX,2015,1,77.0,7.0,37,JAX,9,NaN,False
...,...,...,...,...,...,...,...,...,...,...
2725,2024_18_KC_DEN,2024,18,28.0,5.0,1,KC,0,NaN,False
2726,2024_18_SEA_LA,2024,18,70.0,0.0,1,SEA,30,NaN,True
2727,2024_18_LAC_LV,2024,18,70.0,0.0,1,LAC,34,NaN,True
2728,2024_18_MIA_NYJ,2024,18,34.0,10.0,1,MIA,20,NaN,False


In [16]:
avg_temps = pd.read_csv('team_avg_temp.csv')
avg_temps.head()

,team,week_num,avg_temp
0,ARI,1,70.0
1,ARI,2,70.0
2,ARI,3,70.0
3,ARI,4,70.0
4,ARI,5,70.0


In [17]:
# the nflreadpy schedules dataset uses "LA" for the Rams. Replacing with "LAR" for ease of merging
split_teams_df.loc[split_teams_df['focus_team'] == 'LA', 'focus_team'] = 'LAR'

# merge avg temps in and fill missing focus_temp_delta
split_teams_df = pd.merge(split_teams_df, avg_temps, left_on=['focus_team', 'calendar_week'], right_on=['team', 'week_num'], how='left')
split_teams_df.drop(['team', 'week_num'], axis=1, inplace=True)

# TODO Methodology decision to make here. Do we want to take the temp - avg_temp for the home team,
#   or just make the delta 0 for the home team? Here, I'm just assuming zero for the home team, but this can
#   easily be changed by removing the NaN check

# EA made temp - avg_temp for the home team

# TODO Need Oakland, San Diego, St Louis averages? Or we can just get rid of Oakland stats

split_teams_df.loc[split_teams_df['focus_temp_delta'].isna(), 'focus_temp_delta'] = split_teams_df['temp'] - split_teams_df['avg_temp']
split_teams_df[(split_teams_df['week'] == 1) & (split_teams_df['season'] == 2015)].sort_values('game_id').head(8)

,game_id,season,week,temp,wind,calendar_week,focus_team,focus_score,focus_temp_delta,did_win,avg_temp
2462,2015_01_BAL_DEN,2015,1,88.0,13.0,37,BAL,13,15.556939,False,72.443061
10,2015_01_BAL_DEN,2015,1,88.0,13.0,37,DEN,19,22.434455,True,65.565545
2456,2015_01_CAR_JAX,2015,1,77.0,7.0,37,CAR,20,4.275429,True,72.724571
4,2015_01_CAR_JAX,2015,1,77.0,7.0,37,JAX,9,-2.510000,False,79.510000
2463,2015_01_CIN_OAK,2015,1,73.0,9.0,37,CIN,33,3.501154,True,69.498846
11,2015_01_CIN_OAK,2015,1,73.0,9.0,37,OAK,13,NaN,False,NaN
2457,2015_01_CLE_NYJ,2015,1,77.0,8.0,37,CLE,10,8.930097,False,68.069903
5,2015_01_CLE_NYJ,2015,1,77.0,8.0,37,NYJ,31,5.856923,True,71.143077


In [18]:
team_stats_polars = nfl.load_team_stats(seasons_to_load, summary_level="week")
team_stats_df = team_stats_polars.to_pandas()
team_stats_df = team_stats_df[team_stats_df['season_type'] == 'REG']
team_stats_df.columns
team_stats_df = team_stats_df[[
    'season',
    'week',
    'team',
    'attempts',
    'carries',
    'passing_epa',
    'rushing_epa'
]]

In [19]:
# woah, feature engineering ;)

team_stats_df['conventional_plays_ran'] = team_stats_df['attempts'] + team_stats_df['carries']
team_stats_df['pass_to_run_ratio'] = team_stats_df['attempts'] / team_stats_df['carries']
team_stats_df.head()

,season,week,team,attempts,carries,passing_epa,rushing_epa,conventional_plays_ran,pass_to_run_ratio
0,2024,1,ARI,31,25,-1.184850,4.984950,56,1.240000
1,2024,1,ATL,26,22,-11.084853,-6.224773,48,1.181818
2,2024,1,BAL,41,32,-2.491160,8.459208,73,1.281250
3,2024,1,BUF,23,33,9.161098,3.497710,56,0.696970
4,2024,1,CAR,31,20,-18.467295,-8.230615,51,1.550000


In [20]:
split_teams_df['current_franchise_location'] = split_teams_df['focus_team']

split_teams_df.loc[split_teams_df['focus_team'] == 'OAK', 'current_franchise_location'] = 'LV'
split_teams_df.loc[split_teams_df['focus_team'] == 'SD', 'current_franchise_location'] = 'LAC'
split_teams_df.loc[split_teams_df['focus_team'] == 'STL', 'current_franchise_location'] = 'LAR'
team_stats_df.loc[team_stats_df['team'] == 'LA', 'team'] = 'LAR'


final_df = pd.merge(split_teams_df, team_stats_df, left_on=['current_franchise_location', 'season', 'week'], right_on=['team', 'season', 'week'], how='left')
final_df.drop(['current_franchise_location', 'team'], axis=1, inplace=True)
final_df.head()

,game_id,season,week,temp,wind,calendar_week,focus_team,focus_score,focus_temp_delta,did_win,avg_temp,attempts,carries,passing_epa,rushing_epa,conventional_plays_ran,pass_to_run_ratio
0,2015_01_PIT_NE,2015,1,65.0,7.0,37,NE,28,-1.093469,True,66.093469,32,24,15.961351,-0.252627,56,1.333333
1,2015_01_IND_BUF,2015,1,56.0,15.0,37,BUF,27,-9.687551,True,65.687551,19,36,9.972638,-2.539012,55,0.527778
2,2015_01_GB_CHI,2015,1,72.0,11.0,37,CHI,23,2.529143,False,69.470857,36,33,0.229273,7.547915,69,1.090909
3,2015_01_KC_HOU,2015,1,70.0,0.0,37,HOU,20,0.000000,False,70.000000,47,21,-6.469611,-4.144401,68,2.238095
4,2015_01_CAR_JAX,2015,1,77.0,7.0,37,JAX,9,-2.510000,False,79.510000,40,21,-15.522407,-0.896787,61,1.904762


In [21]:
final_df.to_csv("combined_stats.csv", index=False)

In [113]:
!pip freeze > team_stats_requirements.txt